In [1]:
### >> Multilayer Perceptron << ###
## http://deeplearning.net/tutorial/mlp.html ##

import os
import sys

import theano
import theano.tensor as T
import numpy as np

from logistic import load_data,LogisticRegression

In [2]:
# Define the hidden layer class
class HiddenLayer(object):
    def __init__(self,rng,input,n_in,n_out,w=None,b=None):
        self.input = input
        if w is None:
            wval = np.asarray(rng.uniform(low=-np.sqrt(6. / (n_in + n_out)),
                    high=np.sqrt(6. / (n_in + n_out)),
                    size=(n_in, n_out)),
                              dtype = theano.config.floatX)
            w = theano.shared(wval,name='w',borrow=True)
        #bias
        if b is None:
            b = theano.shared(
                value = np.zeros( (n_out,),dtype=theano.config.floatX),
                name='b',
                borrow=True
            )
        
        #b_values = np.zeros((n_out,), dtype=theano.config.floatX)
        #b = theano.shared(value=b_values, name='b', borrow=True)
        
        self.w = w
        self.b = b
        self.output = T.tanh(T.dot(input,self.w)+self.b)
        self.params = [self.w,self.b]    
        

In [3]:
datasets = load_data('mnist.pkl.gz')

batch_size = 20

train_set_x, train_set_y = datasets[0]
valid_set_x, valid_set_y = datasets[1]
test_set_x, test_set_y = datasets[2]

# compute number of minibatches for training, validation and testing
n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size

... loading data


In [4]:
### : Testing the hidden layer ###
#------------------------------##

x = T.matrix('x')

rng = np.random.RandomState(1234)
hl = HiddenLayer(rng, input=x, n_in=28*28, n_out=500)

In [5]:
### Define MLP class ###
class MLP(object):
    def __init__(self,rng,input,n_in,n_h,n_out):
        self.hidden_layer = HiddenLayer(rng,input=input,n_in=n_in,n_out=n_h)
        self.output_layer = LogisticRegression(input=self.hidden_layer.output, n_in=n_h,n_out=n_out)
        #regularization
        self.L1 = abs(self.hidden_layer.w).sum() + abs(self.output_layer.w).sum()
        self.L2 = (self.hidden_layer.w**2).sum() + (self.output_layer.w**2).sum()
        # Negative Log Likelihood
        self.neg_log_likelihood = (self.output_layer.neg_log_likelihood)
        # errors function
        self.errors = (self.output_layer.errors)
        # params
        self.params = self.hidden_layer.params + self.output_layer.params
        
        self.input = input

In [6]:
index = T.lscalar('index')
x = T.matrix('x')
y = T.ivector('y')
rng = np.random.RandomState(1234)

# instantiate MLP classifier
cl = MLP(rng, input = x, n_in = 28*28, n_h = 500, n_out = 10)

In [7]:
# setup cost
cost = cl.neg_log_likelihood(y) + (cl.L1 * 0.00) + (cl.L2 * 0.0001)

# setup gradient
gparams = [ T.grad(cost,param) for param in cl.params ]

# setup updates 
updates = [ (param, param - 0.01*gparam) for param,gparam in zip(cl.params,gparams)]

In [8]:
# compile training function
train = theano.function(inputs=[index],
                       outputs=cost,
                       updates=updates,
                       givens = { x : train_set_x[index * batch_size : (index+1)*batch_size],
                                  y : train_set_y[index * batch_size : (index+1)*batch_size]
                                }
                       )

In [ ]:
# Actual training begins here
minibatch_avg_cost = 0
for j in xrange(100):
    for i in xrange(n_train_batches):
        minibatch_avg_cost = train(i)
    print 'iteration ',j,' : cost : ', minibatch_avg_cost
    

iteration  0  : cost :  0.754471752453
iteration  1  : cost :  0.612203956742
iteration  2  : cost :  0.533654163437
iteration  3  : cost :  0.48214325281
iteration  4  : cost :  0.444899943415
iteration  5  : cost :  0.415430178061
iteration  6  : cost :  0.390241678427
iteration  7  : cost :  0.367454908955
iteration  8  : cost :  0.346179195053
iteration  9  : cost :  0.326149753193
iteration  10  : cost :  0.307374123556
iteration  11  : cost :  0.289896807167
iteration  12  : cost :  0.273731653996
iteration  13  : cost :  0.258860937642
iteration  14  : cost :  0.245239175748
iteration  15  : cost :  0.232795365595
iteration  16  : cost :  0.221439796364
iteration  17  : cost :  0.21107495932
iteration  18  : cost :  0.201605584501
iteration  19  : cost :  0.192944655183
iteration  20  : cost :  0.185015434102
iteration  21  : cost :  0.177750989743
iteration  22  : cost :  0.171092671691
iteration  23  : cost :  0.164988446585
iteration  24  : cost :  0.159391510685
iteration  2